# Data Quality 

**Under Construction** 

Suppose you are confronted with a new dataset in your company. Here we learn:

*   How to explore a new dataset   
*   How to measure the quality of the new dataset
*   How to handle low quality datapoints

*Data often contain errors that may cause incorrect data analysis results.*

*We distinguish between inliers and outliers, where inliers are values inside the normal feature distribution and outliers are outside.*

*Outliers should be considered with caution because sometimes data contain unusual but correct data that represent valuable information and should not be considered as error. Example: Earthquake events in seismographich records.*

**2-sigma rule**: We define an **outlier** $x_k^{(i)}$ as a data point, that deviates from ther mean by at least twice the standard deviation
    
$$|x_k^{(i)} - \bar{x}^{(i)} | > 2 \cdot s^{(i)}$$

with the standard deviation of the i^th feature:

$$s^{(i)} = \sqrt{\frac{1}{n-1} \sum_{k = 1}^{n} \left( x_k^{(i)} - \bar{x}^{(i)} \right)^2}$$

*If sufficient data are available and data quality is important, then suspicious data may be completely removed.*

**Note:** Also for μ-σ-standardization outliers should be removed before, they
may otherwise strongly affect feature means and standard deviations.

# Setup

In [1]:
# Common imports
import numpy as np
import os
import pandas as pd

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "data_quality"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

# Introduction

Preparing and cleaning data is a crucial aspect of building a machine-learning model. 

# First Steps with a new dataset

## Load the data

In [3]:
df = pd.read_csv('complete.csv', 
                 usecols = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 
                 low_memory=False)

Explanation:
   *    **low_memory**: lower memory use while parsing
   *    **usecols**: use subset of the columns

## Take a look at the data (over and over again)

In [4]:
df.head(3)

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667


## Get the shape of the data

In [5]:
print('The data set consists of {} columns and {} rows.'.format(df.shape[0], df.shape[1]))

The data set consists of 88875 columns and 11 rows.


## Get the variable types

In [6]:
df.dtypes

datetime                 object
city                     object
state                    object
country                  object
shape                    object
duration (seconds)       object
duration (hours/min)     object
comments                 object
date posted              object
latitude                 object
longitude               float64
dtype: object

## Get a quick description of the data

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88875 entries, 0 to 88874
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   datetime              88875 non-null  object 
 1   city                  88679 non-null  object 
 2   state                 81356 non-null  object 
 3   country               76314 non-null  object 
 4   shape                 85757 non-null  object 
 5   duration (seconds)    88873 non-null  object 
 6   duration (hours/min)  85772 non-null  object 
 7   comments              88749 non-null  object 
 8   date posted           88875 non-null  object 
 9   latitude              88875 non-null  object 
 10  longitude             88875 non-null  float64
dtypes: float64(1), object(10)
memory usage: 7.5+ MB


## Dig deeper into your dataset

### Datetime

In [8]:
df['datetime'] = pd.to_datetime(df['datetime']) #, format='%d/%m/%Y %H%M')

ParserError: hour must be in 0..23: 10/10/2005 24:00

Go and ask Stack Overflow!

In [9]:
df['datetime']   = pd.to_datetime(df.datetime.str.replace('24:','00:'))
df['datetime_2'] = df['datetime'].dt.strftime('%Y-%m-%d %H:%M') 

In [10]:
df[['datetime', 'datetime_2']].dtypes

datetime      datetime64[ns]
datetime_2            object
dtype: object

In [11]:
df.dtypes

datetime                datetime64[ns]
city                            object
state                           object
country                         object
shape                           object
duration (seconds)              object
duration (hours/min)            object
comments                        object
date posted                     object
latitude                        object
longitude                      float64
datetime_2                      object
dtype: object

In [12]:
print('First timestamp of dataset is {} and last timestamp is {}.'.format(min(df.datetime), max(df.datetime)))

First timestamp of dataset is 1906-11-11 00:00:00 and last timestamp is 2014-05-08 18:45:00.


In [13]:
(df.groupby('datetime').size() > 1).value_counts()

False    67370
True      8670
dtype: int64

In [14]:
(df.groupby('datetime_2').size() > 1).value_counts()

False    67370
True      8670
dtype: int64

In [16]:
df_dupl = (df.groupby('datetime_2').size() > 1).to_frame()

In [17]:
df_dupl.columns = ['# of entries']

In [18]:
df_dupl[df_dupl['# of entries'] == True].index

Index(['1943-08-15 00:00', '1944-01-01 12:00', '1945-06-01 12:00',
       '1947-07-01 20:00', '1947-07-15 15:00', '1947-07-15 21:00',
       '1948-06-01 00:00', '1949-09-15 21:00', '1950-06-15 20:00',
       '1950-06-30 17:30',
       ...
       '2014-05-01 21:15', '2014-05-01 21:30', '2014-05-01 23:00',
       '2014-05-02 22:00', '2014-05-03 21:00', '2014-05-03 22:00',
       '2014-05-03 22:30', '2014-05-06 22:00', '2014-05-06 23:00',
       '2014-05-07 20:30'],
      dtype='object', name='datetime_2', length=8670)

In [19]:
df[df.datetime == '1943-08-15 00:00']#.shape[0]

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,datetime_2
73312,1943-08-15,unknown,la,NaN,light,900,10-15 minutes,1942 Louisiana midnight light like a...,2/14/2008,37.245443,-107.827839,1943-08-15 00:00
73313,1943-08-15,unknown,la,NaN,unknown,720,10-12 minutes,White light at midnight traveling into outer s...,1/30/2014,37.245443,-107.827839,1943-08-15 00:00


In [20]:
# %timeit 
df_dupl = df.loc[df.datetime.isin(df.groupby('datetime').size() > 1).index.to_list()]

In [21]:
df_dupl.head(2)

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,datetime_2
0,1949-10-10 20:30:00,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111,1949-10-10 20:30
1,1949-10-10 21:00:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082,1949-10-10 21:00


## Create a test set

### Random Sampling

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
train_set, test_set = train_test_split(df, test_size = 0.2, random_state = 42)

In [25]:
train_set.head(2)

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,datetime_2
25764,2006-01-27 01:00:00,armagh (republic of ireland),NaN,NaN,unknown,1200,20 minutes,the lights in our town were too bright for any...,2/14/2006,0,0.000000,2006-01-27 01:00
53455,2011-06-12 22:56:00,hudson falls,ny,us,other,600,10 minutes,Spiral ufo seen in hudson falls.,6/20/2011,43.3005556,-73.586389,2011-06-12 22:56


This is simple **random sampling**.

### Stratified Sampling

In [26]:
df.city.value_counts()

seattle                   570
phoenix                   488
portland                  399
las vegas                 395
los angeles               375
                         ... 
raynham (northeast of)      1
castries (st. lucia)        1
harris ranch                1
ledbetter                   1
beaverton/tigard            1
Name: city, Length: 22018, dtype: int64